# QUESTION 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
results = requests.get(url).text
results

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"YCZxryWJ-aR9i0D4-0QRRgAAAIw","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1004901187,"wgRevisionId":1004901187,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","Po

In [4]:
s = BeautifulSoup(results, "xml")

In [5]:
table = s.find('table')

In [6]:
df = pd.DataFrame(columns=["PostalCode", "Borough",  "Neighborhood"])
df

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []

In [7]:
for i in table.find_all("tr"):
    data = []
    for td in i.find_all('td'):
        data.append(td.text.strip())
    if len(data) == 3:
        df.loc[len(df)] = data

df.head()

PostalCode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

In [8]:
df = df[df["Borough"] != "Not assigned"]
df.head()

PostalCode           Borough                                 Neighborhood
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [9]:
df = df[df["Neighborhood"] != "Not assigned"]
df.head()

PostalCode           Borough                                 Neighborhood
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [10]:
#although postcodes do not seem duplicated, i will still make process to be sure.

df_new=df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_new

PostalCode
M1B                                       Malvern, Rouge
M1C               Rouge Hill, Port Union, Highland Creek
M1E                    Guildwood, Morningside, West Hill
M1G                                               Woburn
M1H                                            Cedarbrae
                             ...                        
M9N                                               Weston
M9P                                            Westmount
M9R    Kingsview Village, St. Phillips, Martin Grove ...
M9V    South Steeles, Silverstone, Humbergate, Jamest...
M9W                  Northwest, West Humber - Clairville
Name: Neighborhood, Length: 103, dtype: object

In [11]:
df_new=df_new.reset_index(drop=False)
df_new

PostalCode                                       Neighborhood
0          M1B                                     Malvern, Rouge
1          M1C             Rouge Hill, Port Union, Highland Creek
2          M1E                  Guildwood, Morningside, West Hill
3          M1G                                             Woburn
4          M1H                                          Cedarbrae
..         ...                                                ...
98         M9N                                             Weston
99         M9P                                          Westmount
100        M9R  Kingsview Village, St. Phillips, Martin Grove ...
101        M9V  South Steeles, Silverstone, Humbergate, Jamest...
102        M9W                Northwest, West Humber - Clairville

[103 rows x 2 columns]

In [12]:
df_new.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
df_new

PostalCode                                Neighborhood_joined
0          M1B                                     Malvern, Rouge
1          M1C             Rouge Hill, Port Union, Highland Creek
2          M1E                  Guildwood, Morningside, West Hill
3          M1G                                             Woburn
4          M1H                                          Cedarbrae
..         ...                                                ...
98         M9N                                             Weston
99         M9P                                          Westmount
100        M9R  Kingsview Village, St. Phillips, Martin Grove ...
101        M9V  South Steeles, Silverstone, Humbergate, Jamest...
102        M9W                Northwest, West Humber - Clairville

[103 rows x 2 columns]

In [13]:
df = pd.merge(df, df_new, on='PostalCode')

In [14]:
df

PostalCode           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
..         ...               ...   
98         M8X         Etobicoke   
99         M4Y  Downtown Toronto   
100        M7Y      East Toronto   
101        M8Y         Etobicoke   
102        M8Z         Etobicoke   

                                          Neighborhood  \
0                                            Parkwoods   
1                                     Victoria Village   
2                            Regent Park, Harbourfront   
3                     Lawrence Manor, Lawrence Heights   
4          Queen's Park, Ontario Provincial Government   
..                                                 ...   
98       The Kingsway, Montgomery Road, Old Mill North   
99                                Church and Wellesley   
100  Business reply mail Processing Centre, South C...   
101  Old Mill South, King's Mill Park, Sunnylea, Hu...   
102  Mimico NW, The Queensway West, South of Bloor,...   

                                   Neighborhood_joined  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4          Queen's Park, Ontario Provincial Government  
..                                                 ...  
98       The Kingsway, Montgomery Road, Old Mill North  
99                                Church and Wellesley  
100  Business reply mail Processing Centre, South C...  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  
102  Mimico NW, The Queensway West, South of Bloor,...  

[103 rows x 4 columns]

In [15]:
df.drop("Neighborhood",axis=1,inplace=True)

In [16]:
df.drop_duplicates(inplace=True)

In [17]:
df.rename(columns={"Neighborhood_joined": "Neighborhood"},inplace=True)
df.head()

PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [18]:
df.shape

(103, 3)